# Vector Search and RAG function application based on SuperDuperDB

In [ ]:
import os
import click
from tqdm import tqdm

import sentence_transformers
from dotenv import load_dotenv
from superduper import (
    Document,
    Listener,
    model,ObjectModel,
    Schema,
    VectorIndex,
    superduper,
    vector
)
# from superduper.backends.mongodb import
import superduper_mongodb
load_dotenv()

## Connect to mongodb database

In [2]:
mongodb_uri = os.getenv("MONGODB_URI", "superduperdb-demo")
artifact_store = os.getenv("ARTIFACT_STORE", "data/artifact_store")

db = superduper(mongodb_uri, artifact_store=f"filesystem://{artifact_store}")

2024-Sep-02 23:33:00.22| INFO     | localhost.localdomain| superduper.base.build:56   | Data Client is ready. MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, serverselectiontimeoutms=5000)
2024-Sep-02 23:33:00.22| INFO     | localhost.localdomain| superduper.base.build:35   | Connecting to Metadata Client with engine:  MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, serverselectiontimeoutms=5000)
2024-Sep-02 23:33:00.23| INFO     | localhost.localdomain| superduper.backends.local.artifacts:29   | Creating artifact store directory
2024-Sep-02 23:33:00.23| INFO     | localhost.localdomain| superduper.base.build:141  | Connecting to compute client: Compute(uri=None, compute_kwargs={}, _path='superduper.backends.local.compute.LocalComputeBackend')
2024-Sep-02 23:33:00.23| INFO     | localhost.localdomain| superduper.base.datalayer:106  | Building Data Layer
2024-Sep-02 23:33:00.23| INFO     | localhost.locald

In [3]:
 db.show()

[]

## Parse pdf files and store them in the database

In [4]:
from superduper.ext.unstructured.encoder import unstructured_encoder

db.add(unstructured_encoder)

pdf_folder = 'pdf-folders'

pdf_paths = [os.path.join(pdf_folder, pdf) for pdf in os.listdir(pdf_folder)]
# collection = superduper_mongodb("source")
to_insert = [
    Document({"elements": unstructured_encoder(pdf_path)}) for pdf_path in pdf_paths
]
# db.execute(collection.insert_many(to_insert))
# _ = db['source'].insert_many(to_insert).execute()
db['source'].insert_many(to_insert).execute()

2024-Sep-02 23:33:07.27| WARNING  | localhost.localdomain| superduper.misc.annotations:119  | add is deprecated and will be removed in a future release.
2024-Sep-02 23:33:07.31| WARNING  | localhost.localdomain| superduper.base.document:390  | Leaf dill already exists
2024-Sep-02 23:33:07.36| INFO     | localhost.localdomain| superduper.backends.mongodb.data_backend:226  | Table source does not exist, auto creating...
2024-Sep-02 23:33:36.92| WARNING  | localhost.localdomain| superduper.base.document:390  | Leaf dill already exists
2024-Sep-02 23:33:37.54| WARNING  | localhost.localdomain| superduper.base.document:390  | Leaf dill already exists
2024-Sep-02 23:33:37.54| WARNING  | localhost.localdomain| superduper.backends.local.artifacts:82   | File data/volvo/artifacts/b43cdc0a4ad627315455f82b0c4337324af85268 already exists
2024-Sep-02 23:33:37.54| WARNING  | localhost.localdomain| superduper.backends.local.artifacts:82   | File data/volvo/artifacts/5f43436f588988dd524b44301f3dc556ac

([ObjectId('66d691a29930a382816c9b10'), ObjectId('66d691a29930a382816c9b11')],
 None)

In [5]:
db.show()

[{'type_id': 'datatype', 'identifier': 'unstructured'}]

In [ ]:
db['source'].find_one().execute().unpack()

## Create a chunking model to chunk pdf chunks

In [6]:
def merge_metadatas(metadatas, return_center=False):
    MAX_NUM = 999999999
    if not metadatas:
        return {}
    p1, p2, p3, p4 = (MAX_NUM, MAX_NUM), (MAX_NUM, 0), (0, 0), (0, MAX_NUM)
    for metadata in metadatas:
        p1_, p2_, p3_, p4_ = metadata["coordinates"]["points"]
        p1 = (min(p1[0], p1_[0]), min(p1[1], p1_[1]))
        p2 = (min(p2[0], p2_[0]), max(p2[1], p2_[1]))
        p3 = (max(p3[0], p3_[0]), max(p3[1], p3_[1]))
        p4 = (max(p4[0], p4_[0]), min(p4[1], p4_[1]))
    points = (p1, p2, p3, p4)
    if return_center:
        points = {"x": (p1[0] + p3[0]) / 2, "y": (p1[1] + p3[1]) / 2}
        page_number = metadata["page_number"]
    return {"points": points, "page_number": page_number}


def create_chunk_and_metadatas(page_elements, stride=3, window=10):
    datas = []
    for i in range(0, len(page_elements), stride):
        windown_elements = page_elements[i : i + window]
        metadatas = [e.metadata.to_dict() for e in windown_elements]
        chunk = "\n".join([e.text for e in windown_elements])
        datas.append(
            {"txt": chunk, "metadata": merge_metadatas(metadatas, return_center=True)}
        )
    return datas


@model(flatten=True, model_update_kwargs={'document_embedded': False})
def get_chunks(elements):
    from collections import defaultdict

    pages_elements = defaultdict(list)
    for element in elements:
        pages_elements[element.metadata.page_number].append(element)

    all_chunks_and_links = sum(
        [
            create_chunk_and_metadatas(page_elements)
            for _, page_elements in pages_elements.items()
        ],
        [],
    )
    return all_chunks_and_links


In [7]:
MODEL_IDENTIFIER_CHUNK = "chunk"
upstream_listener= Listener(
        model=get_chunks,
        select=db['source'].select(),
        key="elements",
       uuid=MODEL_IDENTIFIER_CHUNK
)
db.apply(upstream_listener)

2024-Sep-02 23:33:59.97| INFO     | localhost.localdomain| superduper.jobs.queue:87   | Running jobs for listener.chunk with ids: [ObjectId('66d691a29930a382816c9b10'), ObjectId('66d691a29930a382816c9b11')]
2024-Sep-02 23:34:00.01| INFO     | localhost.localdomain| superduper.backends.local.compute:58   | Submitting job. function:<function method_job at 0x7fe0577440e0>
2024-Sep-02 23:34:00.02| INFO     | localhost.localdomain| superduper.components.model:672  | Requesting prediction in db - [get_chunks] with predict_id chunk
Using select {'_base': '?source-find', '_builds': {'source-find': {'_path': 'superduper.backends.mongodb.query.parse_query', 'documents': [], 'query': 'source.find()'}}, '_blobs': {}, '_files': {}} and ids [ObjectId('66d691a29930a382816c9b10'), ObjectId('66d691a29930a382816c9b11')]
2024-Sep-02 23:34:01.29| INFO     | localhost.localdomain| superduper.components.model:797  | Adding 2 model outputs to `db`
2024-Sep-02 23:34:02.51| SUCCESS  | localhost.localdomain| su

(['listener.chunk'],
 Listener(identifier='chunk', uuid='chunk', upstream=None, plugins=None, key='elements', model=ObjectModel(identifier='get_chunks', uuid='1a30e1c5-ef02-43f9-9b98-1d125d17806d', upstream=None, plugins=None, signature='*args,**kwargs', datatype=None, output_schema=None, flatten=True, model_update_kwargs={'document_embedded': False}, predict_kwargs={}, compute_kwargs={}, validation=None, metric_values={}, num_workers=0, object=<function get_chunks at 0x7fe01f9d9da0>), select=source.find(), active=True, predict_kwargs={}))

In [8]:
db.show()

[{'type_id': 'datatype', 'identifier': 'unstructured'},
 {'type_id': 'listener', 'identifier': 'chunk'},
 {'type_id': 'model', 'identifier': 'get_chunks'}]

In [9]:
upstream_listener.outputs_key

'_outputs.chunk'

In [ ]:
# MODEL_IDENTIFIER_CHUNK = "chunk"
# from superduper import ObjectModel
# chunk_model = ObjectModel(
#     identifier=MODEL_IDENTIFIER_CHUNK,
#     object=get_chunks,
#     flatten=True,
#     model_update_kwargs={"document_embedded": False},
#     output_schema=Schema(identifier="myschema", fields={"txt": "string"}),
# )

# db.add(
#     Listener(
#         model=chunk_model,
#         select=select,
#         key="elements",
#     )
# )
# upstream_listener= Listener(
#         model=get_chunks,
#         select=db['source'].find(),
#         key="elements",
#        uuid=MODEL_IDENTIFIER_CHUNK
# )
# db.apply(upstream_listener)

## Embedding all text blocks and building vector indexes

In [ ]:
SOURCE_KEY = "elements"
MODEL_IDENTIFIER_EMBEDDING = "embedding"
VECTOR_INDEX_IDENTIFIER = "vector-index"
# COLLECTION_NAME_CHUNK = f"_outputs.{SOURCE_KEY}.{MODEL_IDENTIFIER_CHUNK}"
COLLECTION_NAME_CHUNK = f"_outputs.{MODEL_IDENTIFIER_CHUNK}" # _outputs.chunk
# CHUNK_OUTPUT_KEY = f"_outputs.{SOURCE_KEY}.{MODEL_IDENTIFIER_CHUNK}"
CHUNK_OUTPUT_KEY = f"_outputs.{MODEL_IDENTIFIER_CHUNK}.txt"
indexing_key = upstream_listener.outputs_key # Same as CHUNK_OUTPUT_KEY
chunk_collection = db[COLLECTION_NAME_CHUNK]

def preprocess(x):
    if isinstance(x, dict):
        # For model chains, the logic of this key needs to be optimized.
        chunk = sorted(x.items())[-1][1]
        return chunk["txt"]
    return x
from superduper_sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer(
    identifier=MODEL_IDENTIFIER_EMBEDDING,
    object=sentence_transformers.SentenceTransformer("BAAI/bge-large-en-v1.5", device="cuda"),
    datatype=vector(shape=(1024,)),
    device="cuda",
    # predict_method="encode",
    # preprocess=preprocess,
    postprocess=lambda x: x.tolist(),
    # batch_predict=True,
    predict_kwargs={"show_progress_bar": True},
    # device='cuda'

)
# Create vector-index
db.apply(
    VectorIndex(
        VECTOR_INDEX_IDENTIFIER,
        indexing_listener=Listener(
            select=chunk_collection.select(),
            key=CHUNK_OUTPUT_KEY,  # Key for the documents
            # key=indexing_key,  # Key for the documents
            model=embedding_model,  # Specify the model for processing
            # predict_kwargs={"max_chunk_size": 64},
        )
    )
)

In [ ]:
print(len(embedding_model.predict("What is superduper")))

In [12]:
db.show()

[{'type_id': 'datatype', 'identifier': 'unstructured'},
 {'type_id': 'listener', 'identifier': '268aed10-bd67-4b22-a9fd-5145b6b06c7e'},
 {'type_id': 'listener', 'identifier': 'c3183b22-644e-4154-a53c-76e2876c1b60'},
 {'type_id': 'listener', 'identifier': 'chunk'},
 {'type_id': 'model', 'identifier': 'embedding'},
 {'type_id': 'model', 'identifier': 'get_chunks'},
 {'type_id': 'vector_index', 'identifier': 'vector-index'}]

In [31]:
db[COLLECTION_NAME_CHUNK].find_one().execute().unpack()


{'_id': ObjectId('66d691ba9930a382816c9b16'),
 '_outputs': {'chunk': {'txt': "DEAR VOLVO OWNER,\nCongratulations on your new truck and thank you for your vote of confidence! We hope that you will derive great satisfaction and benefit from your truck for many years to come.\nThis Driver Guide contains information tailored to your particular truck. It describes the truck's equipment, care and maintenance, and gives tips for safe and fuel-efficient driving.\nYou have the Driver Guide app in the side display in your truck. There is also an app that can be downloaded to Android and Apple devices. You find the app at your device's app store. The Driver Guide is available in web format at the following address: https://driverguide.volvotrucks.com\nTo get direct access to the Driver Guide for your truck, scan the QR code.\nIf you have any questions or want to know more about your truck, please contact your authorised Volvo dealer.\n©2012 Volvo Trucks, Göteborg.\nReproduction of the contents of

## Define a vector search function

In [13]:
from pprint import pprint
def vector_search(query, top_k=5):
    collection = db[COLLECTION_NAME_CHUNK]
    out = db.execute(
        collection.like(
            Document({CHUNK_OUTPUT_KEY: query}),
            vector_index=VECTOR_INDEX_IDENTIFIER,
            n=top_k,
        ).select({})
    )

    if out:
        out = sorted(out, key=lambda x: x['score'], reverse=True)
    for r in out:
        score = r["score"]
        # chunk_data = r.outputs("elements", "chunk")
        chunk_data = r['_outputs.chunk'] # upstream_listener.outputs_key
        metadata = chunk_data["metadata"]
        chunk_message = {}
        chunk_message["score"] = score
        chunk_message["metadata"] = metadata
        txt = chunk_data["txt"]
        print(txt)
        print()
        print(chunk_message)
        print("\n\n", '-' * 20)

In [14]:
vector_search("What is the function of keys 10 to 12 on the left steering wheel keypad?")

2024-Sep-02 23:40:35.86| INFO     | localhost.localdomain| superduper.base.datalayer:905  | {}


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

NOTE
For the sake of road safety, it is advised that you primarily use voice control (if available) or the steering wheel keypads when driving.
Steering wheel keypads
Keys 10 and 11 are used for phone calls. The others are used for navigating in the displays and controlling the infotainment system. The function of each key is the following:
1 Navigate left.
AA 338361
2 Navigate up.
3 Navigate right.
4 Navigate down.
5 Select.

{'score': 0.8308685339539639, 'metadata': {'points': {'x': 169.35586999999998, 'y': 225.92667}, 'page_number': 267}}


 --------------------
wheel is locked in its new position.
Steering wheel keypads
Left-hand keypad
Keys 10 to 12 control the audio in the truck. The others control cruise control or adaptive cruise control. The function of each key is the following:
1 Resume the previously set speed.
2 Increase speed.
3 Decrease speed.
4 Select current speed as set speed.
5 Switch off cruise control or adaptive
cruise control.

{'score': 0.7987961896635563, 'meta

## Define an LLM model

In [15]:
MODEL_IDENTIFIER_LLM = "llm"
prompt_template = (
    "The following is a document and question about the volvo user manual\n"
    "Only provide a very concise answer\n"
    "{context}\n\n"
    "Here's the question:{input}\n"
    "answer:"
)

# from superduper.ext.vllm import VllmModel
from superduper_vllm import VllmModel
from superduper.ext.openai import OpenAIChatCompletion

llm = VllmModel(
    identifier=MODEL_IDENTIFIER_LLM,
    model_name="TheBloke/Mistral-7B-Instruct-v0.2-AWQ",
    prompt_func=prompt_template,
    vllm_kwargs={ 
        "gpu_memory_utilization": 0.50,
        "max_model_len": 2048,
        "quantization": "awq"
                   },
    predict_kwargs={"max_tokens": 1024, "temperature": 0.8},
)
# Add the llm instance

db.apply(llm)

([],
 VllmModel(identifier='llm', uuid='6cec26fd-2cc2-4bd0-9610-75ced2c1d162', upstream=None, plugins=None, signature='singleton', datatype=None, output_schema=None, flatten=False, model_update_kwargs={}, predict_kwargs={'max_tokens': 1024, 'temperature': 0.8}, compute_kwargs={}, validation=None, metric_values={}, num_workers=0, prompt='{input}', prompt_func="The following is a document and question about the volvo user manual\nOnly provide a very concise answer\n{context}\n\nHere's the question:{input}\nanswer:", max_batch_size=4, model_name='TheBloke/Mistral-7B-Instruct-v0.2-AWQ', tensor_parallel_size=1, trust_remote_code=True, vllm_kwargs={'gpu_memory_utilization': 0.5, 'max_model_len': 2048, 'quantization': 'awq', 'tensor_parallel_size': 1, 'trust_remote_code': True, 'model': 'TheBloke/Mistral-7B-Instruct-v0.2-AWQ'}, on_ray=False, ray_address=None, ray_config={}))

In [16]:
db.show()

[{'type_id': 'datatype', 'identifier': 'unstructured'},
 {'type_id': 'listener', 'identifier': '268aed10-bd67-4b22-a9fd-5145b6b06c7e'},
 {'type_id': 'listener', 'identifier': 'c3183b22-644e-4154-a53c-76e2876c1b60'},
 {'type_id': 'listener', 'identifier': 'chunk'},
 {'type_id': 'model', 'identifier': 'embedding'},
 {'type_id': 'model', 'identifier': 'get_chunks'},
 {'type_id': 'model', 'identifier': 'llm'},
 {'type_id': 'vector_index', 'identifier': 'vector-index'}]

## Define a QA function

In [32]:
from IPython.display import Markdown
from IPython.display import display
import pandas as pd
def qa(query, vector_search_top_k=5):
    collection = db[COLLECTION_NAME_CHUNK]
    output, out = db.execute(
        model_name=MODEL_IDENTIFIER_LLM,
        query=query,
        context_select=collection.like(
            Document({CHUNK_OUTPUT_KEY: query}),
            vector_index=VECTOR_INDEX_IDENTIFIER,
            n=vector_search_top_k,
        ).select({}),
        context_key=f"{CHUNK_OUTPUT_KEY}.0.txt",
    )
    if out:
        out = sorted(out, key=lambda x: x["score"], reverse=True)
    page_messages = []
    for source in out:
        chunk_data = source.outputs("elements", "chunk")
        metadata = chunk_data["metadata"]
        page_number = metadata["page_number"]
        points = metadata["points"]
        score = source["score"]
        page_messages.append(
            {"page_number": page_number, "points": points, "score": score}
        )
    df = pd.DataFrame(page_messages)
    display(output.content)
    display(df)

In [33]:
qa("What is the function of keys 10 to 12 on the left steering wheel keypad?")

AttributeError: 'str' object has no attribute 'type'